# Plotting reflectivity from NCEP grib2 data

In [ ]:
import numpy as np
import xarray as xr
import pygrib
from pyproj import CRS
from pathlib import Path
import datetime as dt
from scipy.ndimage import gaussian_filter
import metpy.calc as mpcalc
from metpy.units import units
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
plt.rcParams['savefig.facecolor']='white'

In [ ]:
model_name = 'arw2'
model_dir = Path(f'../data/models/{model_name}/')
save_dir = f'plots/{model_name}/'

Now that we know our model name we can get available initialization dates and times. My data is archived with folders for each day, with the path to the files being: model/YYYYmmdd/file.grib2.

In [ ]:
# loop through files and display available times here
init_dates = [dt.datetime.strptime(d.name,'%Y%m%d') \
              for d in list(model_dir.iterdir())]
print(f'available initialization dates:')
if len(init_dates) > 0:
    for date in init_dates:
        init_times = set([f.name.split(".")[1].replace('z',"").replace('t',"") \
                          for f in list((model_dir/f'{date:%Y%m%d}').glob("*.grib2"))])
        for t in init_times:
            print(f'{date:%Y %m %d} {t}z')
else:
    print('no data found')

We can now pick from these available dates by assigning the year, month day and hour variables. Here I pick the 2022/07/19 initialization.

In [ ]:
year = 2022
month = 7
day = 19
init_hour = 12
forecast_step = 12

Now we provide the boundaries to plot

In [ ]:
# Using latitude in decimal degrees N
north = 58 
south = 48 
# Using longitude in decimal degrees W
east = -110 
west = -118
# Need this later for plotting
bounds = dict(north=north,south=south,east=east,west=west)

We need to provide keyword arguments to xarray for parsing grib2 files. These arguments will not necessarily work for data coming from a different center (such as CMC or ECMWF)

In [ ]:
# shortname for reflectivity:
varname = 'refd' 
# "levtype" specifies type of level (isobaric, isentropic, ect...)
levtype = 'heightAboveGround' 
# 1000 m above ground level:
level = 1000 
# kwargs to pass to xarray for parsing:
grib_kwargs = {'shortName' : varname, 'typeOfLevel' : levtype,'level' : level} 

Now we are ready to load the grib2 file using xarray. 

In [ ]:
folder = model_dir / f"{year}{month}{day}/"
file = [str(f) for f in list(folder.glob('*grib2')) if \
        f't{init_hour}z'in f.name and f'f{forecast_step}' in f.name][0]
print(f"retrieving file: {file}")
ds = xr.open_dataset(file,engine = 'cfgrib', \
                     backend_kwargs = dict(filter_by_keys = grib_kwargs), \
                     decode_coords = 'all') 

In [ ]:
# Useful functions for mapping grib2 files

def get_crs(file):
    grib = pygrib.open(file)
    msg = grib.message(1)
    cf_params = CRS(msg.projparams).to_cf()
    grib.close()
    return cf_params

def deg_w_to_e(lon):
    return ((lon + 180) % 360) - 180


cf_params = get_crs(file)
ds = ds.metpy.assign_crs(cf_params).metpy.assign_y_x()
ds = ds.assign(longitude = deg_w_to_e(ds.longitude))
# Note: here I padded the data with an extra 15 deg of lat/lon to make sure no areas will
#       have missing data 
ds = ds.where(((ds.latitude < north+15) & (ds.latitude > south-15) \
               & (ds.longitude > west-15) & (ds.longitude < east+15)),drop=True)

In [ ]:
# Here is a nice colormap for plotting radar data
def radar_colormap():
    reflectivity_colors = [
    "#00ecec", # 5
    "#01a0f6", # 10
    "#0000f6", # 15
    "#00ff00", # 20
    "#00c800", # 25
    "#009000", # 30
    "#ffff00", # 35
    "#e7c000", # 40
    "#ff9000", # 45
    "#ff0000", # 50
    "#d60000", # 55
    "#c00000", # 60
    "#ff00ff", # 65
    "#9955c9", # 70
    "#808080"  # 75
    ]
    cmap = colors.ListedColormap(reflectivity_colors)
    return cmap

refl_range = np.arange(5,80,5) 
cmap= radar_colormap() 

Set up the figure and add cartopy features such as coast outlines and state borders

In [ ]:
fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1,projection=ccrs.LambertConformal())
ax.set_extent((west,east,south,north))

res = '50m'
ax.add_feature (cfeature.LAND.with_scale(res),zorder = 2)
ax.add_feature (cfeature.OCEAN.with_scale(res),zorder = 2)
ax.add_feature(cfeature.COASTLINE.with_scale(res),zorder = 2)
ax.add_feature (cfeature.LAKES.with_scale(res), alpha = 0.5,zorder = 2)
ax.add_feature (cfeature.STATES.with_scale(res),zorder = 2)

In [ ]:
# Add some cities to the plot here using their lats/lons
cities = {'calgary' : [51.045833, -114.0575], 'edmonton' : [53.53444,-113.490278], \
          'red deer' : [52.268056,-113.811111]}

for name,coords in cities.items():
    
    ax.plot(coords[1],coords[0], marker = 'o',color = 'black',\
            markersize = 3,alpha = .8,transform =  ccrs.PlateCarree(),zorder = 2)
    
    ax.text(coords[1]+.2,coords[0],name.title(),fontsize = 6,alpha = .8, \
            transform = ccrs.PlateCarree(),zorder = 2)

In [ ]:
cf = ax.contourf(ds.longitude,ds.latitude,ds[varname],extend = 'max', \
                 levels=refl_range,cmap=cmap,transform=ccrs.PlateCarree(),zorder = 3)

cbar = plt.colorbar(cf,extend = 'max',ax=ax,ticks = np.arange(5,85,10),\
                    fraction=0.046, pad=0.03,shrink=0.5)

cbar.ax.tick_params(labelsize=10)
cbar.ax.set_ylabel("Reflectivity (dBZ)",fontsize=10)
title = ax.set_title(f"model: {model_dir.stem}\nSimulated Reflectivity and Updraft Helicity Contours\nInit: {init:%Y-%m-%d %H%M}z\nValid: {valid:%Y-%m-%d %H%M}z",fontsize=8,loc = 'left')
# remove comment below to save the figure
# plt.savefig([PATH TO SAVE FOLDER].png,dpi = 600,bbox_inches = 'tight')
plt.show()